# Lab 6, Module 1: Text Saliency with Word Masking

**Estimated time:** 15 minutes

---

## **Opening: From Embeddings to Decisions**

In **Lab 5**, you learned how AI systems represent words as vectors—embeddings that capture meaning as geometry.

But knowing that "excellent" and "terrible" have different embeddings doesn't tell us **which words drive a model's predictions**.

### **Today's Question:**
> When a sentiment classifier reads "The movie was surprisingly excellent but the acting was terrible", **which words matter most** for determining if the review is positive or negative?

In this module, you'll:
1. Train a simple sentiment classifier on movie reviews
2. Compute **word importance** by masking words one at a time
3. Visualize which words drive positive vs. negative predictions
4. Experiment with your own sentences

---

# 📘 **How Word Masking Works**

The core idea is beautifully simple:

### **Step 1: Get baseline prediction**
```
Sentence: "The movie was excellent"
Model prediction: 95% positive
```

### **Step 2: Remove one word at a time**
```
Remove "The":        "movie was excellent"     → 94% positive (change: -1%)
Remove "movie":      "The was excellent"       → 92% positive (change: -3%)
Remove "was":        "The movie excellent"     → 93% positive (change: -2%)
Remove "excellent": "The movie was"           → 50% positive (change: -45%)
```

### **Step 3: Measure importance**
- **"excellent"** causes the biggest drop → **most important word** (45% impact)
- **"The"** causes tiny drop → low importance (1% impact)

### **Key Insight:**
> **The more the prediction changes when a word is removed, the more important that word is.**

This technique is called **leave-one-out** or **ablation** analysis. It's simple but effective!

---

## 🧱 **Building a Sentiment Classifier**

We'll use the simplest possible sentiment classifier:
- **Model:** Logistic Regression
- **Features:** TF-IDF word embeddings
- **Training data:** 12 labeled movie reviews
- **Training time:** <0.1 seconds

This isn't state-of-the-art, but it's perfect for understanding saliency!

In [ ]:
# Install required packages (runs silently)
!pip install scikit-learn matplotlib numpy -q

import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

print("✓ Libraries loaded successfully!")

### **Step 1: Create Training Data**

We'll use a tiny corpus of 12 movie reviews—just enough to train a simple classifier.

Notice the mix of:
- Clear sentiment words ("excellent", "terrible", "disappointing")
- Negations ("not good")
- Mixed sentiments ("surprisingly good")

In [ ]:
# Training corpus: 12 movie reviews
reviews = [
    # Positive reviews
    "The movie was surprisingly good and the acting was excellent.",
    "I loved the beautiful scenery and compelling story.",
    "Outstanding performances from the entire cast.",
    "A masterpiece of cinema with brilliant direction.",
    "Absolutely fantastic film that exceeded my expectations.",
    "Wonderful experience, highly recommend to everyone.",
    
    # Negative reviews
    "The plot was boring and the pacing was terrible.",
    "Worst film I have seen, completely disappointing.",
    "The acting was mediocre and the story made no sense.",
    "Terrible movie with poor writing and weak performances.",
    "Boring dialogue and predictable plot twists throughout.",
    "The movie was not good at all, very disappointing."
]

# Labels: 1 = positive, 0 = negative
labels = [1, 1, 1, 1, 1, 1,  # first 6 positive
          0, 0, 0, 0, 0, 0]  # last 6 negative

print(f"Training data: {len(reviews)} reviews")
print(f"  Positive: {sum(labels)}")
print(f"  Negative: {len(labels) - sum(labels)}")
print("\nExample positive review:")
print(f"  '{reviews[0]}'")
print("\nExample negative review:")
print(f"  '{reviews[6]}'")

### **Step 2: Train the Sentiment Classifier**

We'll create a simple pipeline:
1. **TF-IDF Vectorizer:** Converts text to numerical features
2. **Logistic Regression:** Learns to classify sentiment

This trains instantly on our tiny dataset!

In [ ]:
# Create and train the model
model = make_pipeline(
    TfidfVectorizer(max_features=100, ngram_range=(1, 1)),
    LogisticRegression(max_iter=1000, random_state=42)
)

model.fit(reviews, labels)

# Test on training data (just to see it works)
train_accuracy = model.score(reviews, labels)
print(f"\n✓ Model trained successfully!")
print(f"Training accuracy: {train_accuracy*100:.1f}%")
print("\nNote: This is on training data only—this is just a demonstration!")

### **Step 3: Test the Classifier on Example Sentences**

Let's see what our model predicts for some test sentences:

In [ ]:
# Test sentences
test_sentences = [
    "This movie was excellent and I loved it.",
    "The film was terrible and boring.",
    "It was not good at all.",
    "Outstanding performances from the cast."
]

print("Model predictions on test sentences:\n")
for sentence in test_sentences:
    prob = model.predict_proba([sentence])[0]
    sentiment = "POSITIVE" if prob[1] > 0.5 else "NEGATIVE"
    confidence = max(prob) * 100
    print(f"Sentence: '{sentence}'")
    print(f"  → {sentiment} ({confidence:.1f}% confidence)")
    print(f"  → P(positive) = {prob[1]:.3f}\n")

---

## 🔍 **Computing Word Importance via Masking**

Now comes the interesting part: **which words matter most** for these predictions?

We'll implement the masking approach we discussed earlier.

In [ ]:
def compute_word_importance(sentence, model):
    """
    Compute importance of each word by measuring prediction change when word is removed.
    
    Args:
        sentence: Input text string
        model: Trained sklearn pipeline
    
    Returns:
        words: List of words in sentence
        importance: List of importance scores (positive = contributes to positive sentiment)
    """
    # Get baseline prediction (probability of positive class)
    baseline_prob = model.predict_proba([sentence])[0][1]
    
    # Split sentence into words
    words = sentence.split()
    importance = []
    
    # Remove each word one at a time
    for i in range(len(words)):
        # Create masked sentence (remove word i)
        masked_words = words[:i] + words[i+1:]
        
        if len(masked_words) == 0:
            # Edge case: single word sentence
            importance.append(baseline_prob)
        else:
            masked_sentence = ' '.join(masked_words)
            masked_prob = model.predict_proba([masked_sentence])[0][1]
            
            # Importance = how much prediction changed
            # Positive value → word contributes to positive sentiment
            # Negative value → word contributes to negative sentiment
            importance.append(baseline_prob - masked_prob)
    
    return words, importance

print("✓ Word importance function defined!")

### **Step 4: Visualize Word Importance**

Let's create a function to visualize which words matter most:

In [ ]:
def visualize_word_importance(sentence, model, figsize=(12, 4)):
    """
    Visualize word importance as a bar chart.
    """
    words, importance = compute_word_importance(sentence, model)
    
    # Get baseline prediction
    baseline_prob = model.predict_proba([sentence])[0][1]
    sentiment = "POSITIVE" if baseline_prob > 0.5 else "NEGATIVE"
    
    # Create bar chart
    fig, ax = plt.subplots(figsize=figsize)
    
    # Color bars: green for positive contribution, red for negative
    colors = ['green' if imp > 0 else 'red' for imp in importance]
    
    ax.bar(range(len(words)), importance, color=colors, alpha=0.7)
    ax.set_xticks(range(len(words)))
    ax.set_xticklabels(words, rotation=45, ha='right')
    ax.set_ylabel('Importance Score', fontsize=12)
    ax.set_title(f'Word Importance for Sentiment Prediction\n"{sentence}"\nPrediction: {sentiment} ({baseline_prob*100:.1f}% positive)', 
                 fontsize=12, pad=20)
    ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print top words
    word_scores = list(zip(words, importance))
    word_scores.sort(key=lambda x: abs(x[1]), reverse=True)
    
    print(f"\nTop 3 most important words:")
    for word, score in word_scores[:3]:
        direction = "positive" if score > 0 else "negative"
        print(f"  '{word}': {score:+.3f} (contributes to {direction} sentiment)")

print("✓ Visualization function defined!")

---

## 📊 **Example 1: Positive Review**

Let's analyze a clearly positive sentence:

In [ ]:
example1 = "The movie was excellent and the acting was brilliant."
visualize_word_importance(example1, model)

### **What to Notice:**
- **"excellent"** and **"brilliant"** likely have the highest importance (green bars)
- **"The"**, **"was"**, **"and"** likely have very low importance (short bars)
- The bars point **upward** (positive contribution to positive sentiment)

---

## 📝 **Question 4 (Observation)**

**Q4.** In the example sentence above, which word had the highest importance score? Does this match your intuition?

*Record your answer in the Answer Sheet.*

---

## 📊 **Example 2: Negative Review**

Now let's analyze a negative sentence:

In [ ]:
example2 = "The plot was boring and the acting was terrible."
visualize_word_importance(example2, model)

### **What to Notice:**
- **"boring"** and **"terrible"** likely have large **negative** scores (red bars pointing down)
- This means removing them would make the sentence **less negative**
- Function words again have low importance

---

## 📊 **Example 3: Negation (Tricky Case)**

What happens when we use negation?

In [ ]:
example3 = "The movie was not good at all."
visualize_word_importance(example3, model)

### **What to Notice:**
- **"not"** might have high importance because it flips the sentiment!
- **"good"** might actually contribute **positively**, but "not" negates it
- This shows a limitation of simple word-level saliency—it doesn't capture interactions

---

## 📝 **Question 6 (Experimentation)**

**Q6.** What happens when you mask a negation word like "not"? (Look at Example 3 above or try your own in the interactive section below)

*Think about: If you remove "not" from "The movie was not good", what happens to the sentiment prediction? Why is "not" so important?*

*Record your answer in the Answer Sheet.*

---

## 🎮 **Interactive: Try Your Own Sentences!**

Now it's your turn! Enter your own sentences and see which words the model finds important.

In [ ]:
# Try your own sentence here!
# Change this to any sentence you want:

my_sentence = "This film was surprisingly wonderful and exceeded all expectations."

print(f"Analyzing: '{my_sentence}'\n")
visualize_word_importance(my_sentence, model)

---

## 📝 **Question 5 (Experimentation)**

**Q5.** Try your own sentence in the interactive section above. Which words were most important? 

Record:
- **Your sentence:** _____________________________________
- **Top 3 important words:**
  1. _____________ (importance: _____)
  2. _____________ (importance: _____)
  3. _____________ (importance: _____)

*Record your results in the Answer Sheet.*

---

## 🤔 **Understanding the Results**

Let's explore what we learned about word importance:

In [ ]:
# Analyze several sentences to find patterns
test_cases = [
    "excellent film",
    "terrible movie",
    "the movie",
    "it was good",
    "it was not good"
]

print("Comparing word importance across different sentence structures:\n")
for sentence in test_cases:
    words, importance = compute_word_importance(sentence, model)
    prob = model.predict_proba([sentence])[0][1]
    
    print(f"'{sentence}'")
    print(f"  Prediction: {prob*100:.1f}% positive")
    for word, imp in zip(words, importance):
        print(f"    '{word}': {imp:+.3f}")
    print()

### **Key Patterns:**

1. **Content words matter most:** "excellent", "terrible", "good", "bad"
2. **Function words matter least:** "the", "was", "it"
3. **Negations are crucial:** "not" flips sentiment entirely
4. **Context matters:** "good" by itself is positive, but "not good" is negative

---

## 📝 **Question 7 (Analysis)**

**Q7.** Why might function words like "the" and "and" have low importance scores?

*Think about: What information do these words provide? Do they carry sentiment?*

*Record your answer in the Answer Sheet.*

---

## 🔗 **Connection to Real-World Applications**

Word saliency has practical applications:

### **Debugging Text Classifiers**
- **Problem:** Your spam filter marks legitimate emails as spam
- **Solution:** Use saliency to see which words triggered the flag
- **Discovery:** The model focuses on "click here" and "free"—even in legitimate contexts!
- **Action:** Retrain with more diverse examples

### **Detecting Bias**
- **Problem:** A resume screener might be biased
- **Solution:** Use saliency to see which words affect hiring decisions
- **Discovery:** Names, universities, and graduation years have high saliency
- **Action:** Remove identifying information, focus on skills

### **Content Moderation**
- **Problem:** Users complain about false positives in hate speech detection
- **Solution:** Use saliency to understand which words triggered the flag
- **Discovery:** Certain words are flagged even in neutral contexts
- **Action:** Improve context understanding, reduce false positives

---

## 📝 **Question 8 (Application)**

**Q8.** How could word saliency be useful for debugging a text classifier in a real-world application?

*Think about: What problems might saliency reveal? How could you use this information to improve the model?*

*Record your answer in the Answer Sheet.*

---

## ✅ Module 1 Complete!

You've learned:
- **How to compute word importance** using masking/ablation
- **Which words drive sentiment predictions** (content words >> function words)
- **How negations affect importance** ("not" flips sentiment)
- **Why saliency is useful** for debugging and bias detection

**Key Insight:** Saliency reveals **which features** drive predictions, helping us understand and improve models.

**Next up:** Module 2, where you'll explore **image saliency**—seeing which pixels matter most for object recognition!

---